In [74]:
import pymorphy2
import re
import string
import gensim
import pandas as pd
from pymorphy2 import MorphAnalyzer
from collections import Counter
from nltk.corpus import stopwords
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
morph = MorphAnalyzer()

In [17]:
stops = set(stopwords.words('russian')) | {'gt', 'int', 'if', 'x', 'to', 'in', 'return', 'case', 'i', '•', 
                                          'null', 'set', 'as', 'from', 'var', 'lt', 'on', 'is', 'end', 'b',
                                          'else', 'bool', 'const', 'function', 'class', 'string', 'import',
                                          'then', 'type', 'person', 'select', 'def', 'up', 'down', 'active',
                                          'print', 'assert', 'j', 'str', 'loop', 'char', 'unsigned', 'true',
                                          'main', 'func'}

In [18]:
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [19]:
texts = open('/Users/macbookpro/Downloads/habr.txt').read().splitlines()

In [20]:
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

In [34]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.7) # threshold можно подбирать

In [35]:
p = gensim.models.phrases.Phraser(ph)

In [36]:
ngrammed_texts = p[texts]

In [37]:
words = gensim.corpora.Dictionary(texts)

In [38]:
words.filter_extremes(no_above=0.15, no_below=22)
words.compactify()

In [39]:
print(words)

Dictionary(6770 unique tokens: ['2-х', '3.0', 'address', 'api', 'architecture']...)


In [62]:
tfidf = gensim.models.TfidfModel(corpus, id2word=words)

In [63]:
corpus = tfidf[corpus]

In [40]:
corpus = [words.doc2bow(text) for text in texts]

In [41]:
lda = gensim.models.LdaMulticore(corpus, 100, id2word=words, eval_every=0)

In [42]:
lda.print_topics()

[(41,
  '0.007*"google" + 0.004*"изображение" + 0.003*"карта" + 0.003*"android" + 0.003*"игрок" + 0.003*"переменный" + 0.003*"глаз" + 0.002*"вирус" + 0.002*"microsoft" + 0.002*"письмо"'),
 (81,
  '0.004*"игра" + 0.004*"обучение" + 0.003*"модуль" + 0.003*"new" + 0.003*"диск" + 0.002*"qt" + 0.002*"контейнер" + 0.002*"counter" + 0.002*"частота" + 0.002*"битый"'),
 (85,
  '0.006*"000" + 0.003*"модуль" + 0.003*"пакет" + 0.003*"рубль" + 0.003*"метка" + 0.002*"виртуальный" + 0.002*"свет" + 0.002*"вызов" + 0.002*"цвета" + 0.002*"кнопка"'),
 (53,
  '0.004*"переменный" + 0.004*"датчик" + 0.003*"частота" + 0.003*"id" + 0.003*"сигнал" + 0.002*"тест" + 0.002*"звук" + 0.002*"компонент" + 0.002*"игра" + 0.002*"кнопка"'),
 (4,
  '0.004*"нейрон" + 0.003*"public" + 0.003*"бизнес" + 0.003*"value" + 0.003*"сотрудник" + 0.003*"false" + 0.003*"new" + 0.003*"сигнал" + 0.003*"обучение" + 0.003*"виртуальный"'),
 (83,
  '0.006*"игра" + 0.003*"google" + 0.003*"игрок" + 0.002*"учёный" + 0.002*"канал" + 0.002*"pub

In [43]:
lda = gensim.models.LdaMulticore(corpus, 200, id2word=words, passes=3, alpha='asymmetric', eval_every=0)

In [44]:
lda.print_topics()

[(199,
  '0.007*"studio" + 0.007*"окно" + 0.006*"visual" + 0.006*"иконка" + 0.004*"программирование" + 0.003*"плагин" + 0.003*"камера" + 0.003*"оборудование" + 0.003*"ребёнок" + 0.003*"paypal"'),
 (198,
  '0.011*"connected" + 0.010*"российский" + 0.008*"компонент" + 0.008*"react" + 0.008*"message" + 0.004*"site" + 0.004*"виртуальный" + 0.004*"native" + 0.003*"export" + 0.003*"apple"'),
 (197,
  '0.026*"azure" + 0.021*"docker" + 0.015*"сборка" + 0.009*"windows" + 0.008*"linux" + 0.008*"run" + 0.007*"репозиторий" + 0.006*"агент" + 0.006*"build" + 0.005*"service"'),
 (195,
  '0.018*"спутник" + 0.016*"марс" + 0.013*"аппарат" + 0.013*"орбита" + 0.012*"земля" + 0.011*"космический" + 0.011*"луна" + 0.010*"планета" + 0.008*"поверхность" + 0.007*"солнечный"'),
 (196,
  '0.025*"тест" + 0.012*"тестирование" + 0.006*"номер" + 0.006*"тестовый" + 0.005*"вызов" + 0.004*"температура" + 0.003*"шанс" + 0.003*"маршрутизация" + 0.003*"вредоносный" + 0.003*"изображение"'),
 (194,
  '0.010*"университет" + 0

In [46]:
lda = gensim.models.LdaModel(corpus, 150, id2word=words, passes=6, alpha='auto', eval_every=0)

In [47]:
lda.print_topics()

[(97,
  '0.206*"лаборатория" + 0.166*"account" + 0.110*"log" + 0.074*"security" + 0.037*"верх" + 0.030*"injection" + 0.022*"подготовиться" + 0.022*"data" + 0.019*"блокироваться" + 0.015*"ноябрь"'),
 (58,
  '0.242*"биржа" + 0.225*"московский" + 0.058*"кредитный" + 0.057*"близость" + 0.051*"400" + 0.033*"zero" + 0.027*"торговый" + 0.021*"доллар" + 0.020*"магнит" + 0.019*"стратегический"'),
 (63,
  '0.255*"days" + 0.097*"подсеть" + 0.057*"cisco" + 0.055*"адрес" + 0.048*"inc" + 0.045*"маска" + 0.032*"connection" + 0.020*"участник" + 0.019*"94" + 0.019*"50"'),
 (66,
  '0.133*"чёрный" + 0.098*"пятница" + 0.080*"золото" + 0.076*"праздник" + 0.057*"openstack" + 0.046*"четверг" + 0.046*"посылка" + 0.039*"т.е" + 0.029*"понедельник" + 0.027*"виртуализация"'),
 (93,
  '0.138*"azure" + 0.098*"microsoft" + 0.072*"vpn" + 0.028*"облако" + 0.027*"team" + 0.024*"виртуальный" + 0.023*"европа" + 0.022*"update" + 0.020*"продолжение" + 0.018*"т.к"'),
 (16,
  '0.244*"провайдер" + 0.107*"sla" + 0.084*"дата-це

In [53]:
topics = [lda.print_topics()]

In [60]:
print(topics[0][4], '\n\n', topics[0][10], '\n\n', topics[0][16])

(16, '0.244*"провайдер" + 0.107*"sla" + 0.084*"дата-центр" + 0.046*"беспилотный" + 0.035*"автомобиль" + 0.027*"cdn" + 0.025*"активироваться" + 0.022*"мириться" + 0.020*"нота" + 0.019*"доставка"') 

 (52, '0.040*"заказчик" + 0.030*"услуга" + 0.016*"документ" + 0.016*"защита" + 0.016*"корпоративный" + 0.013*"предприятие" + 0.012*"ожидание" + 0.012*"бизнес" + 0.011*"внедрение" + 0.011*"ит"') 

 (11, '0.007*"прибор" + 0.006*"мощность" + 0.006*"генератор" + 0.006*"тепло" + 0.004*"измерение" + 0.004*"импульс" + 0.004*"производство" + 0.004*"сопротивление" + 0.004*"длительность" + 0.004*"постоянный"')


модели на преобразованном корпусе:

In [66]:
lda = gensim.models.LdaMulticore(corpus, 100, id2word=words, eval_every=0)

In [67]:
lda.print_topics()

[(26,
  '0.003*"нативный" + 0.003*"power" + 0.002*"игра" + 0.002*"визуализация" + 0.002*"массив" + 0.002*"документ" + 0.002*"болезнь" + 0.002*"контекст" + 0.002*"российский" + 0.002*"ноутбук"'),
 (3,
  '0.002*"ручка" + 0.002*"операция" + 0.002*"юпитер" + 0.002*"пост" + 0.002*"аппарат" + 0.002*"анализатор" + 0.002*"devops" + 0.002*"подписка" + 0.002*"javascript" + 0.002*"react"'),
 (89,
  '0.003*"ibm" + 0.003*"виртуальный" + 0.002*"php" + 0.002*"игра" + 0.002*"сергей" + 0.002*"sap" + 0.002*"вм" + 0.002*"книга" + 0.002*"бумажный" + 0.002*"вирус"'),
 (87,
  '0.002*"учёный" + 0.002*"игра" + 0.002*"таблица" + 0.002*"lt;input" + 0.002*"подсветка" + 0.002*"датчик" + 0.002*"техподдержка" + 0.002*"дисплей" + 0.002*"sum" + 0.002*"фото"'),
 (18,
  '0.003*"конструктор" + 0.002*"игра" + 0.002*"менеджер" + 0.002*"звук" + 0.002*"коллекция" + 0.002*"dns" + 0.002*"выделить" + 0.002*"gitlab" + 0.002*"sudo" + 0.002*"временный"'),
 (36,
  '0.004*"игрок" + 0.003*"триггер" + 0.003*"email" + 0.003*"сотрудник

In [68]:
lda = gensim.models.LdaMulticore(corpus, 200, id2word=words, passes=3, alpha='asymmetric', eval_every=0)

In [69]:
lda.print_topics()

[(196,
  '0.170*"робот" + 0.006*"автоматизация" + 0.004*"дрон" + 0.001*"ия" + 0.001*"watson" + 0.001*"общество" + 0.001*"население" + 0.001*"технологический" + 0.001*"сфера" + 0.001*"корпорация"'),
 (194,
  '0.000*"мучительный" + 0.000*"отметка" + 0.000*"капля" + 0.000*"мина" + 0.000*"морской" + 0.000*"ехать" + 0.000*"отверстие" + 0.000*"погружение" + 0.000*"повредить" + 0.000*"душа"'),
 (199,
  '0.000*"мучительный" + 0.000*"отметка" + 0.000*"капля" + 0.000*"мина" + 0.000*"морской" + 0.000*"ехать" + 0.000*"отверстие" + 0.000*"погружение" + 0.000*"повредить" + 0.000*"душа"'),
 (193,
  '0.000*"мучительный" + 0.000*"отметка" + 0.000*"капля" + 0.000*"мина" + 0.000*"морской" + 0.000*"ехать" + 0.000*"отверстие" + 0.000*"погружение" + 0.000*"повредить" + 0.000*"душа"'),
 (198,
  '0.000*"мучительный" + 0.000*"отметка" + 0.000*"капля" + 0.000*"мина" + 0.000*"морской" + 0.000*"ехать" + 0.000*"отверстие" + 0.000*"погружение" + 0.000*"повредить" + 0.000*"душа"'),
 (197,
  '0.000*"мучительный" + 0.

In [70]:
lda = gensim.models.LdaModel(corpus, 150, id2word=words, passes=6, alpha='auto', eval_every=0)

In [71]:
lda.print_topics()

[(148,
  '0.000*"мучительный" + 0.000*"отметка" + 0.000*"капля" + 0.000*"мина" + 0.000*"морской" + 0.000*"ехать" + 0.000*"отверстие" + 0.000*"погружение" + 0.000*"повредить" + 0.000*"душа"'),
 (149,
  '0.000*"мучительный" + 0.000*"отметка" + 0.000*"капля" + 0.000*"мина" + 0.000*"морской" + 0.000*"ехать" + 0.000*"отверстие" + 0.000*"погружение" + 0.000*"повредить" + 0.000*"душа"'),
 (60,
  '0.000*"мучительный" + 0.000*"отметка" + 0.000*"капля" + 0.000*"мина" + 0.000*"морской" + 0.000*"ехать" + 0.000*"отверстие" + 0.000*"погружение" + 0.000*"повредить" + 0.000*"душа"'),
 (84,
  '0.000*"мучительный" + 0.000*"отметка" + 0.000*"капля" + 0.000*"мина" + 0.000*"морской" + 0.000*"ехать" + 0.000*"отверстие" + 0.000*"погружение" + 0.000*"повредить" + 0.000*"душа"'),
 (117,
  '0.000*"мучительный" + 0.000*"отметка" + 0.000*"капля" + 0.000*"мина" + 0.000*"морской" + 0.000*"ехать" + 0.000*"отверстие" + 0.000*"погружение" + 0.000*"повредить" + 0.000*"душа"'),
 (5,
  '0.000*"мучительный" + 0.000*"отмет

In [72]:
lda = gensim.models.LdaModel(corpus, 85, id2word=words, passes=1, alpha='auto', eval_every=0)

In [73]:
lda.print_topics()

[(59,
  '0.008*"callback" + 0.005*"license" + 0.004*"весы" + 0.004*"аналоговый" + 0.004*"развёртывание" + 0.004*"поисковый" + 0.003*"session" + 0.003*"iot" + 0.003*"выдача" + 0.003*"configuration"'),
 (84,
  '0.009*"qualcomm" + 0.005*"post" + 0.005*"звезда" + 0.004*"чип" + 0.003*"астроном" + 0.003*"арифметический" + 0.003*"geektimes" + 0.003*"учёный" + 0.003*"дц" + 0.002*"тп"'),
 (35,
  '0.009*"микросервис" + 0.008*"view" + 0.008*"коллекция" + 0.007*"игра" + 0.005*"рассказ" + 0.003*"мартин" + 0.003*"серия" + 0.003*"рельеф" + 0.002*"контроллер" + 0.002*"дайджест"'),
 (41,
  '0.008*"спрайт" + 0.008*"когнитивный" + 0.008*"сертификат" + 0.007*"vsphere" + 0.005*"devops" + 0.004*"сертификация" + 0.003*"врач" + 0.003*"схд" + 0.003*"gb" + 0.003*"package"'),
 (17,
  '0.089*"iaas" + 0.014*"ит-инфраструктура" + 0.012*"обоснование" + 0.009*"облачный" + 0.006*"доход" + 0.005*"ип" + 0.004*"книга" + 0.003*"флешек" + 0.003*"облако" + 0.003*"зарубежный"'),
 (76,
  '0.008*"подписка" + 0.007*"воспоминани

Первое, что бросилось в глаза после применения tf-idf — это появление множества дубликатов на нескольких вариантах настроек модели, вот таких: 

(148,
  '0.000*"мучительный" + 0.000*"отметка" + 0.000*"капля" + 0.000*"мина" + 0.000*"морской" + 0.000*"ехать" + 0.000*"отверстие" + 0.000*"погружение" + 0.000*"повредить" + 0.000*"душа"')
  
Вероятно, это как-то связано с количеством проходов по корпусу, но до tf-idf этого не было.

При этом сами темы стали более осмысленными — если однозначную тематику угадать все еще можно не всегда, то связи между словами в теме почти всегда можно найти, в отличие от результатов моделей до tf-idf:

(48,
  '0.011*"сотрудник" + 0.010*"стикер" + 0.007*"программирование" + 0.007*"книга" + 0.007*"пообщаться" + 0.007*"бизнес" + 0.006*"дизайн" + 0.006*"дизайнер" + 0.006*"иконка" + 0.006*"коммуникация"') — что-то про визуальную составляющую, UX
  
(69,
  '0.013*"уязвимость" + 0.012*"шифрование" + 0.012*"домен" + 0.012*"вредоносный" + 0.011*"атака" + 0.011*"пароль" + 0.010*"виртуальный" + 0.010*"злоумышленник" + 0.010*"google" + 0.009*"км/ч"') — что-то про безопасность и инфобез

Некоторые из тем ухудшились, например, в более ранней 

(4,
  '0.014*"пациент" + 0.010*"лечение" + 0.010*"клетка" + 0.009*"заболевание" + 0.008*"болезнь" + 0.008*"вирус" + 0.008*"врач" + 0.007*"пакет" + 0.006*"организм" + 0.006*"учёный"')
  
легче понять, о чем речь, нежели в более поздней

(7,
  '0.070*"учёный" + 0.067*"вызов" + 0.044*"вирус" + 0.042*"java" + 0.033*"заболевание" + 0.028*"deepmind" + 0.026*"журнал" + 0.023*"научный" + 0.021*"стек" + 0.015*"геном"')

.

In [75]:
stexts = [' '.join(text) for text in texts]

In [104]:
vectorizer = TfidfVectorizer(max_features=500, min_df=5, max_df=0.7, ngram_range=(1,3))

In [105]:
X = vectorizer.fit_transform(stexts)

In [106]:
model = NMF(n_components=30)

In [107]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [108]:
feat_names = vectorizer.get_feature_names()

In [109]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 то  очень  какой  делать
1 файл  скрипт  пакет  настройка
2 игра  игрок  разработчик  уровень
3 приложение  мобильный  android  разработка
4 lt  gt  gt lt  компонент
5 система  управление  безопасность  оборудование
6 проект  команда  разработчик  разработка
7 устройство  смартфон  производитель  мобильный
8 год  2016  миллион  стать
9 public  new  void  класс
10 компания  сотрудник  рынок  бизнес
11 the  of  and  for
12 код  функция  значение  переменный
13 сервер  виртуальный  машина  настройка
14 сайт  страница  браузер  адрес
15 язык  программирование  программист  книга
16 windows  microsoft  linux  версия
17 дать  запрос  база  база дать
18 элемент  изображение  материал  модель
19 google  android  сервис  api
20 сигнал  связь  канал  передача
21 тест  тестирование  результат  производительность
22 камера  смартфон  видео  цена
23 amp  for  void  gt
24 пользователь  сообщение  сервис  доступ
25 объект  класс  метод  свойство
26 человек  исследование  жизнь  работа
27 процессор 

In [110]:
model.reconstruction_err_

49.62858140486233

In [90]:
vectorizer = TfidfVectorizer(max_features=1500, min_df=8, max_df=0.5, ngram_range=(1,3))

In [91]:
X = vectorizer.fit_transform(stexts)

In [92]:
model = NMF(n_components=30)

In [93]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [94]:
feat_names = vectorizer.get_feature_names()

In [95]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 проект  команда  задача  разработка
1 файл  папка  скрипт  пакет
2 игра  игрок  игровой  играть
3 человек  учёный  мозг  исследование
4 устройство  сигнал  звук  частота
5 lt  gt  lt div  div
6 приложение  android  мобильный  пользователь
7 public  new  void  private
8 сайт  страница  пользователь  реклама
9 человек  книга  какой  что
10 the  of  and  for
11 космический  спутник  орбита  аппарат
12 товар  рубль  цена  скидка
13 виртуальный  машина  виртуальный машина  windows
14 уязвимость  атака  безопасность  пароль
15 процессор  память  intel  диск
16 печать  принтер  3d  материал
17 доклад  конференция  участник  тема
18 функция  объект  значение  метод
19 тест  тестирование  код  тестировать
20 react  css  javascript  js
21 сервер  запрос  база  сервис
22 компания  сотрудник  рынок  бизнес
23 камера  видео  смартфон  изображение
24 бот  сообщение  пользователь  канал
25 дата центр  центр  дата  оборудование
26 вселенная  энергия  звезда  свет
27 amp  amp amp  echo  for
28 сеть  

In [96]:
model.reconstruction_err_

55.30179936400718

In [97]:
vectorizer = TfidfVectorizer(max_features=1200, min_df=21, max_df=0.35, ngram_range=(1,3))

In [98]:
X = vectorizer.fit_transform(stexts)

In [99]:
model = NMF(n_components=30)

In [100]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [101]:
feat_names = vectorizer.get_feature_names()

In [102]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 клиент  продукт  команда  сотрудник
1 файл  папка  скрипт  команда
2 игра  игрок  игровой  играть
3 вселенная  энергия  теория  свет
4 устройство  смартфон  телефон  датчик
5 lt  gt  lt div  div
6 приложение  android  мобильный  ios
7 public  void  new  private
8 windows  microsoft  уязвимость  linux
9 товар  рубль  скидка  цена
10 космический  спутник  аппарат  земля
11 сайт  страница  реклама  браузер
12 рынок  google  российский  миллион
13 the  of  and  for
14 доклад  конференция  участник  тема
15 сервер  запрос  сервис  база
16 учёный  мозг  исследование  клетка
17 печать  принтер  3d  материал
18 объект  класс  метод  свойство
19 язык  программирование  программа  программист
20 сеть  атака  безопасность  трафик
21 дата центр  центр  дата  оборудование
22 бот  сообщение  робот  канал
23 функция  значение  элемент  переменный
24 amp  amp amp  echo  for
25 камера  видео  смартфон  изображение
26 react  css  javascript  js
27 сигнал  звук  частота  напряжение
28 книга  что  какой

In [103]:
model.reconstruction_err_

55.18919946327033

лучшей оказалась первая модель.

In [122]:
top_words = model.components_.argsort()[:,:-5:-1]
ready = []
for i in range(top_words.shape[0]):
    ready.append([feat_names[j] for j in top_words[i]])

In [125]:
print(ready[2], ready[3], ready[6])

['игра', 'игрок', 'разработчик', 'уровень'] ['приложение', 'мобильный', 'android', 'разработка'] ['проект', 'команда', 'разработчик', 'разработка']


осмысленности в резуль